<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Data Visualization Workflow

In this workflow within PyCCAPT, we can crop the data, do the voltage and bowl calibration, calculate the 3d reconstruction, and do the ranging.
</div>

In [ ]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import numpy as np
import subprocess
from ipywidgets import widgets
from IPython.display import display
from ipywidgets import fixed, interact_manual
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration import widgets as wd
from pyccapt.calibration.data_tools import data_tools
from pyccapt.calibration.tutorials.tutorials_helpers import helper_data_loader
from pyccapt.calibration.tutorials.tutorials_helpers import helper_visualization
from pyccapt.calibration.calibration import share_variables
from pyccapt.calibration.calibration import ion_selection

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In case of recieving the error about pytable library, you have to install the pytables library with conda command. to do that you can open a new cell and copy the line below in it. Then just run it like other cells. The pytables library will be innstalled.
    
`!conda install --yes --prefix {sys.prefix} pytables`
</div>


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In the cell below we create variable object. This object is used in many of the functions to share the data between functions in as easy way. 
</div>


In [ ]:
variables = share_variables.Variables()

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
By clicking on the button below, you can select the dataset file you want to crop. The dataset file can be in various formats, including HDF5, EPOS, POS, ATO, and CSV. The cropped data will be saved in the same directory as the original dataset file in a new directory nammed load_crop. The name of the cropped dataset file will be the same as the original dataset file. The figures will be saved in the same directory as the dataset file.
</div>

In [ ]:
button = widgets.Button(description='load dataset')

@button.on_click
def open_file_on_click_r(b):
    global dataset_path
    folder_path = variables.last_directory
    script = '..//..//data_tools//run_dataset_path_qt.py'
    cmd = f"python {script} {folder_path}"
    result = subprocess.run(cmd, capture_output=True, text=True, shell=True)
    dataset_path = result.stdout.strip()
    variables.last_directory = dataset_path

button

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## ROI Selection and Data Cropping
    

</div>



<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

    
From the dropdown lists below, you can select the instrument specifications of the dataset. The instrument specifications are the same as the ones used for the calibration process. Data mode is specify the dataset structure. The dataset can be in raw or calibrated mode. The flight path length is the distance between the sample and the detector. The t0 is the time of flight of the ions with the lowest mass-to-charge ratio. The maximum mass-to-charge ratio is the maximum mass-to-charge ratio of tat you want to plot. You can also change it in te related cells. The detector diameter is the diameter of the detector.
</div>


In [ ]:
# create an object for selection of instrument specifications of the dataset
tdc, pulse_mode, flight_path_length, t0, max_mc, det_diam = wd.dataset_instrument_specification_selection()

# Display lists and comboboxes to selected instrument specifications
display(tdc, pulse_mode, flight_path_length, t0, max_mc)

In [ ]:
helper_data_loader.load_data(dataset_path, max_mc.value, flight_path_length.value, pulse_mode.value, tdc.value, variables, processing_mode=False)
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
display(variables.data)
display(variables.range_data)

<div style="margin: 0 auto; padding: 20px;">

Load a pre saved range file if you want from cell below
</div>


In [ ]:
button_r = widgets.Button(description='load range dataset')

@button_r.on_click
def open_file_on_click_r(b):
    global range_path
    folder_path = variables.last_directory
    script = '..//..//data_tools//run_dataset_path_qt.py'
    cmd = f"python {script} {folder_path}"
    result = subprocess.run(cmd, capture_output=True, text=True, shell=True)
    range_path = result.stdout.strip()

button_r

In [ ]:
if 'range_path' in globals():
    variables.range_data = data_tools.read_range(range_path)

In [ ]:
display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the range in the hdf5 and csv format.

</div>

In [ ]:
interact_manual_range = interact_manual.options(manual_name="save range")
interact_manual_range(data_tools.save_range, variables=fixed(variables));

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the dataset in any other format that you want.

</div>

In [ ]:
interact_manual_data = interact_manual.options(manual_name="save data")
interact_manual_data(data_tools.save_data, data=fixed(variables.data), variables=fixed(variables),
                name=widgets.Text(value=variables.result_data_name),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]),
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]),
               temp=fixed(False));

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
# Visualization
    
</div>

In [ ]:
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

In [ ]:
helper_visualization.call_visualization(variables)

In [ ]:
from pyccapt.calibration.calibration import mc_plot
hist = variables.data['mc (Da)'].to_numpy()
mc_hist = mc_plot.AptHistPlotter(hist[hist < 40], variables)
mc_hist.plot_histogram(bin_width=0.1, label='mc', steps='stepfilled', log=True)
mc_hist.find_peaks_and_widths()
mc_hist.plot_hist_info_legend(loc='right')
mc_hist.plot_line_hist()
mc_hist.manual_background_fit()

In [ ]:
variables.plotly_3d_reconstruction

In [ ]:
from IPython.display import display, HTML
display(HTML(variables.animation_detector_html))